<a href="https://colab.research.google.com/github/suhaib99/car_classification/blob/main/car_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models
from matplotlib import pyplot as plt


In [ ]:
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from torch.utils.data import Dataset
from torchvision import datasets, transforms
np.set_printoptions(6)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# NOTE: uploading the zip file in your drive
# THIS ONE IS FASTER compared to files.upload()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("Open left tab and click files (files icon)")
print("Click drive and navigate to the zip file")
print("Click the 3 dots on file and copy path. Paste path here")
import zipfile
path_to_zip_file = input("File path: ")
# use dot . for cwd
# path_to_extract = input("Path of where you want to store: ")
path_to_extract = "dl4vision-winter-2023-a3"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(path_to_extract)
    # zip_ref.extractall(".") # to current directory

Open left tab and click files (files icon)
Click drive and navigate to the zip file
Click the 3 dots on file and copy path. Paste path here
File path: /content/drive/MyDrive/UC COURSES/599.6 - deep learning for vision/a3/Assignment 3 Dataset.zip


In [ ]:
# THIS ONE MIGHT TAKE TOO LONG OF A TIME, USE ABOVE INSTEAD
from google.colab import files
uploaded = files.upload()
! unzip -qq dl4vision-winter-2023-a3.zip 

KeyboardInterrupt: ignored

Saving FlowerData.zip to FlowerData.zip


In [ ]:
class carData(Dataset):
  
  
  import glob
  from PIL import Image
  def __init__(self, root, transform, ext='.jpg'):
    """ Instantiate an object of the carData
    Args:
      root (str): The root directory includes images, where each class of images is
        inside a separate folder.
      transformation (callable): A image augmentation function to be applied to an image.
        default is None, indicating the there will be no transformation applied
        an image. 
    """
    self.paths = []
    self.labels = []

    # PROBLEM: at folders /test/ or train/ ther are only images
    # so the 2 dictionaries here can be empty
    self.label_encoder = {foldername:idx for idx, foldername in enumerate(os.listdir(root)) if os.path.isdir(os.path.join(root, foldername))}
    self.label_decoder = {idx:foldername for foldername,idx in self.label_encoder.items()}
    # TODO: Read the path to all images and indicat their classes.
    # Hint: You can use glob.glob(f'{directory_path}/**/*.jpg', recursive=True)
    #       to read all images with .jpg file extension inside a directory_path
    #       Also, you can use te self.label_encoder to assign a numerical label
    #       to each flower class
    # Class notes used and went to the professor
    self.transform = transform
  
    def label_image(p):
      if "daisy" in p:
        return 0
      elif "dandelion" in p:
        return 1
      elif "roses" in p:
        return 2
      elif "sunflowers" in p:
        return 3
      else:
        return 4 
    self.paths = glob.glob(f'{root}/**/*.jpg',recursive = True)
    
    for p in self.paths:
      labelToConfirm = label_image(p)
      self.labels.append(labelToConfirm)


  

    # End of TODO
    assert len(self.paths) == len(self.labels), "Number of image paths and labels should be equal"
    

  def __len__(self):
    """ Return the number of samples within the dataset """
    return len(self.paths)

  def __getitem__(self, i):
    """ This method return a tuple made of the i-th element of the dataset, i.e. (image, label).
    
    If the transofrm is not None, you should apply the transform function to the image."""
    #TODO: write code for implementing the requested functionality
    carImagery = Image.open(self.paths[i])
    
    if self.transform:
      carImagery = self.transform(carImagery)
    label = self.labels[i]
    

    return carImagery, label


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

test_dataset = carData("/content/dl4vision-winter-2023-a3/test",transform = None)
print(test_dataset.label_encoder)
print(test_dataset.label_decoder)














{}
{}


In [ ]:
print(len(test_dataset))



4080


In [ ]:
# (A)

import albumentations as A
HEIGHT = 2000
WIDTH = 2000
train_transform = A.Compose(
  transforms=[
        A.Flip(p=0.5),
        A.Rotate(
            limit=(-15, 15),
            interpolation=1,
            border_mode=0,
            value=0,
            mask_value=0,
            always_apply=False,
            p=0,
        ),
        A.ColorJitter(
            brightness=0.7,
            contrast=0.5,
            saturation=0.5,
            hue=0.2,
            always_apply=False,
            p=0.5,
        ),
        A.ChannelShuffle(p=0.9),
        A.RGBShift(r_shift_limit=50, g_shift_limit=50, b_shift_limit=50, p=0.5),
        A.HueSaturationValue(
            hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5
        ),
        A.ToGray(p=0.2),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), always_apply=False, p=0.2),
        A.GaussianBlur(blur_limit=(3, 7), p=0.1),
        A.MotionBlur(blur_limit=(3, 7), p=0.1),
        
    ],
    p=0.5,
)

In [ ]:
# (B) TODO: Using data from the train folder create train_dataset. The train_dataset
#       should use the defined tranformation, i.e. train_transoform to augment
#       each image.
train_dataset = carData('/content/dl4vision-winter-2023-a3/train',transform = train_transform)
print(len(train_dataset))
# there is no validation dataset folder, later one way is to split up train dataset in 2 portions train and validation
validation_dataset = train_dataset



4135


In [ ]:
phases = {
    'train': train_dataset,
    'valid': validation_dataset,
    'test': test_dataset
    }
loader = {
    phase: DataLoader(ds, batch_size=4, shuffle=(phase=='train'))
    for phase, ds in phases.items()
}

In [ ]:
train_dataset[0][0].shape

KeyError: ignored

In [ ]:
def visualize(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(str(label))
  plt.xticks([])
  plt.yticks([])


In [ ]:
train_examples = [train_dataset[i] for i in range(10)]
for image, label in train_examples:
  image = torch.permute(image, (2, 1, 0)).numpy()
  visualize(image, f'Test: {label}')


IndexError: ignored

In [ ]:
def freeze(model):
  for param in model.parameters():
    param.requires_grad = False

# unfreeze the parameters of the first layer
def unfreeze(model):
  for param in model.layer1.parameters():
    param.requires_grad = True

In [ ]:
num_classes = 100  # Flower dataset has 102 classes
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
freeze_features = True
if freeze_features:
  freeze(model)

model.fc = nn.Linear(num_features, num_classes)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader[mode]):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')




In [ ]:
model = model.to(device)
epochs = 20
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  for i, (images, labels) in enumerate(loader['train']):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, loader, device, criterion, mode='valid')

NameError: ignored

In [ ]:
total_params = sum(
	param.numel() for param in model.parameters()
)
print('Total number of parameters:', total_params)

Total number of parameters: 11227812


In [ ]:

trainable_params = sum(
	p.numel() for p in model.parameters() if p.requires_grad
)
print('Total number of trainable parameters:', trainable_params)

Total number of trainable parameters: 51300


In [ ]:
evaluate(model, loader, device, criterion, mode='test')


NameError: ignored

#How do you address model overfitting?